In [31]:
import os
import numpy as np
import pandas as pd
import transformers
import torch
from transformers import TFAutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [2]:
MODEL_EMBED = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"

In [3]:
# Embed layer
tokenizer_EMBED = AutoTokenizer.from_pretrained(MODEL_EMBED)
model_EMBED = AutoModelForSequenceClassification.from_pretrained(MODEL_EMBED)

C:\Users\Mapraw\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [28]:
# body layer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_BODY = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_BODY = AutoModelForSeq2SeqLM.from_pretrained('mrm8488/t5-base-finetuned-question-generation-ap')

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [5]:
model_EMBED.deberta.embeddings.word_embeddings

Embedding(251000, 768, padding_idx=0)

In [37]:
# model_EMBED

In [35]:
model_BODY.shared = model_EMBED.deberta.embeddings.word_embeddings
model_BODY.encoder.embed_tokens = model_BODY.shared
model_BODY.decoder.embed_tokens = model_BODY.shared
model_BODY.lm_head = torch.nn.Linear(in_features=768, out_features=251000, bias=False)

In [40]:
# model_BODY

In [39]:
from datasets import load_dataset

dataset = load_dataset("iapp_wiki_qa_squad")

Reusing dataset iapp_wiki_qa_squad (C:\Users\Mapraw\.cache\huggingface\datasets\iapp_wiki_qa_squad\iapp_wiki_qa_squad\1.0.0\c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
train_dataset  = load_dataset('iapp_wiki_qa_squad', split="train")
valid_dataset = load_dataset('iapp_wiki_qa_squad', split="validation")
test_dataset = load_dataset("iapp_wiki_qa_squad", split="test")

Reusing dataset iapp_wiki_qa_squad (C:\Users\Mapraw\.cache\huggingface\datasets\iapp_wiki_qa_squad\iapp_wiki_qa_squad\1.0.0\c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)
Reusing dataset iapp_wiki_qa_squad (C:\Users\Mapraw\.cache\huggingface\datasets\iapp_wiki_qa_squad\iapp_wiki_qa_squad\1.0.0\c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)
Reusing dataset iapp_wiki_qa_squad (C:\Users\Mapraw\.cache\huggingface\datasets\iapp_wiki_qa_squad\iapp_wiki_qa_squad\1.0.0\c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)


In [46]:
def edit_ans_to_ds(dataset):
#     ans_list = [ans_dict["text"][0] for ans_dict in dataset["answers"]]
    dataset["ans_list"] = dataset["answers"]["text"][0]
#     dataset.add_column("ans_list", ans_list)
    return dataset

In [47]:
train_dataset = train_dataset.map(edit_ans_to_ds)
train_dataset

Loading cached processed dataset at C:\Users\Mapraw\.cache\huggingface\datasets\iapp_wiki_qa_squad\iapp_wiki_qa_squad\1.0.0\c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3\cache-2b7478fb1c11af1c.arrow


Dataset({
    features: ['question_id', 'article_id', 'title', 'context', 'question', 'answers', 'ans_list'],
    num_rows: 5761
})

In [90]:
def create_ans_context(dataset):
    answer = dataset["ans_list"]
    context = dataset["context"]
    input_format = tokenizer_EMBED("answer: %s  context: %s </s>" % (answer, context))
    labels = tokenizer_EMBED(dataset["question"])
    dataset["input_ids"] = input_format["input_ids"]
    dataset["attention_mask"] = input_format["attention_mask"]
    dataset["labels"] = labels["input_ids"]
    return dataset

In [91]:
train_ds_processed = train_dataset.map(create_ans_context)

  0%|          | 0/5761 [00:00<?, ?ex/s]

In [92]:
train_ds_processed = train_ds_processed.remove_columns(['question_id', 'article_id', 'title', 'context', 'question', 'answers', 'ans_list'])

In [93]:
train_ds_processed.set_format("torch")

In [94]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_ds_processed, shuffle=True, batch_size=1)
# eval_dataloader = DataLoader(train_ds, batch_size=1)

In [95]:
from torch.optim import AdamW
optimizer = AdamW(model_BODY.parameters(), lr=5e-5)

In [96]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [97]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_BODY.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(251000, 768, padding_idx=0)
  (encoder): T5Stack(
    (embed_tokens): Embedding(251000, 768, padding_idx=0)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=Fa

In [98]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model_BODY.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_BODY(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/17283 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.83 GiB already allocated; 0 bytes free; 2.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [99]:
batch

{'input_ids': tensor([[     1,  20955,    268,    260,  42275,  81126,   6197,  21750, 111244,
           49948, 142320,   3946,   2092,  21750,  45174,  20280,  10229,  40439,
           18395,   1025,    260, 106744,  16913,    261,  17959,    261,    260,
          230751,  19731,    268,  74303,  58474,    260, 178994, 133828,   4389,
           20649,  30008,    260,  42275,  81126,   6197,    275,  76390,  67865,
           95321, 170505,    268,  44335,    263,    442, 136454,  71495,    504,
             272,  21192, 172336, 148602,  94732, 212304,  43203, 123724,  76390,
           67865,  95321, 170505,  50428,  16913,    261,  17959,    261,    260,
          234005,    323,    260, 230751, 100200,   5491, 224762,  64878,  17857,
          217291, 148602,  94732, 212304,  43203, 123724,  76390,  67865,  95321,
          170505,    260, 148602,  94732, 212304,  43203, 123724,  76390,  67865,
           95321, 170505,  96423,   8008,  12660,  50428, 167510, 135571,   5270,
   